In [1]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [2]:
train_labels = []
train_samples = []

In [3]:
## Create a fake dataset to play with. Imagine a drug trial
for i in range(50):
    random_younger = randint(13,64) #participants with age less than 65
    train_samples.append(random_younger)
    train_labels.append(1) #young with side effects
    
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(0) #old without side effects
    
for i in range(1000):
    random_younger = randint(13,64) #participants with age less than 65
    train_samples.append(random_younger)
    train_labels.append(0) #young without side effects
    
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(1) #old with side effects
    

In [4]:
## Get ready to call Keras models
## a list is not a valid input --> change to np.array

train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels, train_samples = shuffle(train_labels, train_samples)

## Scale the data (to better Gradient Descent performaces)
scaler = MinMaxScaler(feature_range=(0,1))
## Semp r solt cacate : fit_transform si aspetta una certa shape, e quello è il senso di reshape
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))

In [5]:
## Ok, now we have created our fake data. Let's write a simple model that works on them

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [7]:
# Solo se giri con una GPU
# physical_devices = tf.config.experimental.list_physical_devices("GPU")
# print("Num GPUs available: ", len(physical_devices))
# print(physical_devices)
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [8]:
## Ok, let's architect this model
model = Sequential([
    Dense(units=16, input_shape=(1,),activation="relu"),
    Dense(units=32, activation="relu"),
    Dense(units=2, activation="softmax") #output layer (the two possible output class)
    ])

model.summary()

# Notes:
# Dense means fully connected layer
# Nel primo layer devi specificare lo shape dei dati in input. In questo caso noi passiamo (1,).
# Mo che significa (1,) ? Io so che passo un training set, di cui ho fatto il reshape(-1,1) 
print(scaled_train_samples.shape) # --> (2100,1)
# Sostanzialmente, dal vettore colonna che avevo, ora ho una "matrice" di 2100 righe e 1 colonna

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________
(2100, 1)


In [9]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=['accuracy'])

## Loss = cosa stai minimizzando
## metrics = come stai misurando la bontà del modello

In [10]:
model.fit(x=scaled_train_samples, y=train_labels, batch_size=10, epochs=30, shuffle=True, verbose=2)

Epoch 1/30
210/210 - 0s - loss: 0.6830 - accuracy: 0.5371
Epoch 2/30
210/210 - 0s - loss: 0.6607 - accuracy: 0.6143
Epoch 3/30
210/210 - 0s - loss: 0.6331 - accuracy: 0.7195
Epoch 4/30
210/210 - 0s - loss: 0.6052 - accuracy: 0.7438
Epoch 5/30
210/210 - 0s - loss: 0.5756 - accuracy: 0.7738
Epoch 6/30
210/210 - 0s - loss: 0.5451 - accuracy: 0.8033
Epoch 7/30
210/210 - 0s - loss: 0.5140 - accuracy: 0.8338
Epoch 8/30
210/210 - 0s - loss: 0.4836 - accuracy: 0.8500
Epoch 9/30
210/210 - 0s - loss: 0.4548 - accuracy: 0.8633
Epoch 10/30
210/210 - 0s - loss: 0.4280 - accuracy: 0.8719
Epoch 11/30
210/210 - 0s - loss: 0.4038 - accuracy: 0.8867
Epoch 12/30
210/210 - 0s - loss: 0.3824 - accuracy: 0.8957
Epoch 13/30
210/210 - 0s - loss: 0.3638 - accuracy: 0.9014
Epoch 14/30
210/210 - 0s - loss: 0.3477 - accuracy: 0.9100
Epoch 15/30
210/210 - 0s - loss: 0.3338 - accuracy: 0.9133
Epoch 16/30
210/210 - 0s - loss: 0.3221 - accuracy: 0.9186
Epoch 17/30
210/210 - 0s - loss: 0.3118 - accuracy: 0.9186
Epoch 

In [11]:
## Fittare con un validation set
#validation_split=0.1 significa che l'ultimo 10 % del training set è usato come validation_set
model.fit(x=scaled_train_samples, y=train_labels, validation_split=0.1, batch_size=10, epochs=30, shuffle=True, verbose=2)

Epoch 1/30
189/189 - 0s - loss: 0.2533 - accuracy: 0.9360 - val_loss: 0.2893 - val_accuracy: 0.9286
Epoch 2/30
189/189 - 0s - loss: 0.2519 - accuracy: 0.9386 - val_loss: 0.2885 - val_accuracy: 0.9286
Epoch 3/30
189/189 - 0s - loss: 0.2507 - accuracy: 0.9376 - val_loss: 0.2871 - val_accuracy: 0.9286
Epoch 4/30
189/189 - 0s - loss: 0.2497 - accuracy: 0.9402 - val_loss: 0.2855 - val_accuracy: 0.9286
Epoch 5/30
189/189 - 0s - loss: 0.2483 - accuracy: 0.9376 - val_loss: 0.2856 - val_accuracy: 0.9286
Epoch 6/30
189/189 - 0s - loss: 0.2474 - accuracy: 0.9402 - val_loss: 0.2847 - val_accuracy: 0.9333
Epoch 7/30
189/189 - 0s - loss: 0.2465 - accuracy: 0.9402 - val_loss: 0.2836 - val_accuracy: 0.9286
Epoch 8/30
189/189 - 0s - loss: 0.2457 - accuracy: 0.9381 - val_loss: 0.2824 - val_accuracy: 0.9286
Epoch 9/30
189/189 - 0s - loss: 0.2449 - accuracy: 0.9402 - val_loss: 0.2821 - val_accuracy: 0.9286
Epoch 10/30
189/189 - 0s - loss: 0.2442 - accuracy: 0.9376 - val_loss: 0.2817 - val_accuracy: 0.9286

In [13]:
## Ci inventiamo un test set a questo punto
test_labels = []
test_samples = []

for i in range(10):
    random_younger = randint(13,64) #participants with age less than 65
    test_samples.append(random_younger)
    test_labels.append(1) #young with side effects
    
    random_older = randint(65,100)
    test_samples.append(random_older)
    test_labels.append(0) #old without side effects
    
for i in range(200):
    random_younger = randint(13,64) #participants with age less than 65
    test_samples.append(random_younger)
    test_labels.append(0) #young without side effects
    
    random_older = randint(65,100)
    test_samples.append(random_older)
    test_labels.append(1) #old with side effects
    
## Convert to np array data format
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)
test_labels, test_samples = shuffle(test_labels, test_samples)

## MinMax scaling on the reshaped test sample
scaled_test_samples = scaler.fit_transform(test_samples.reshape(-1,1))
    

In [20]:
# Predict
predictions = model.predict(x=scaled_test_samples, batch_size=10, verbose=0)

# for i in predictions:
#     print(i)
#     # print(i[0] + i[1]) # a somma praticamente 1.
    
rounded_preductions = np.argmax(predictions, axis=-1)

In [22]:
## Confusion matrix
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true=test_labels, y_pred=rounded_preductions)

print(cm)

[[194  16]
 [ 10 200]]


In [ ]:
## Save and load the model: ci sono 3 diverse possibilità, ognuna con vantaggi e svantaggi

In [27]:
# 1. model.save()
import os.path
if os.path.isfile("./medical_trail_model.h5") is False:
    model.save("./medical_trail.h5")
    
## Questo metodo è quello più "comprehensive". Salva tutto:
# l'architettura
# pesi
# training configuration: loss, optimizer
# the state of the optimizer: you can resume training exactly where you left off

from tensorflow.keras.models import load_model
new_model = load_model("./medical_trail.h5")
new_model.summary()
new_model.get_weights() #in case you want to inspect the weights
new_model.optimizer

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [32]:
# 2. model.to_json()
json_string = model.to_json()

## Salva soltanto l'architettura del modello, non i pesi, né le sue training configuration

from tensorflow.keras.models import model_from_json
model_architecture = model_from_json(json_string)
model_architecture.summary()
## Ma va ri-trainato, niente è salvato. Serve solo per non perdere l'architettura del modello.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [34]:
# 3. model.save_weights()
import os.path
if os.path.isfile("./my_model_weigths.h5") is False:
    model.save_weights("./my_model_weights.h5")
    
## Siccome questo salva SOLO i pesi, se voglio fare "una prediction", beh, devo rifarmelo
model2 = Sequential([
    Dense(units=16, input_shape=(1,), activation="relu"),
    Dense(units=32, activation="relu"),
    Dense(units=2, activation="softmax")
    ])
model2.load_weights("./my_model_weights.h5") # e gli assegni i pesi che hai salvato prima